In [1]:
import cv2
import matplotlib.pyplot as plt

camera_index = 0


In [35]:
cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW) # this is the magic!


In [3]:

num = 0


imgs = []

while cap.isOpened():

    succes, img = cap.read()

    cv2.imshow('Img',img)

    k = cv2.waitKey(5)

    if k == 27:
        break
    elif k == ord('s'): # wait for 's' key to save and exit
        imgs.append(img)
        print("image saved!")
        num += 1


# Release and destroy all windows before termination
cap.release()

cv2.destroyAllWindows()

image saved!
image saved!
image saved!
image saved!
image saved!
image saved!
image saved!


In [6]:
imgs[0].shape

(480, 640, 3)

In [23]:
import numpy as np
import cv2 as cv
import glob
import pickle


chessboardSize = (5,5)
frameSize = (640,480)



# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

size_of_chessboard_squares_mm = 25
objp = objp * size_of_chessboard_squares_mm


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


In [25]:

for image in imgs:

    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if ret == True:

        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        img = gray.copy()

        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(1000)
    else:
        print("Chessboard not found in image")

cv.destroyAllWindows()

In [ ]:
ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

In [30]:
pickle.dump((cameraMatrix, dist), open( "calibration.pkl", "wb" ))
pickle.dump(cameraMatrix, open( "cameraMatrix.pkl", "wb" ))
pickle.dump(dist, open( "dist.pkl", "wb" ))

### auroco

In [31]:
import cv2.aruco as aruco

In [43]:
import cv2
import cv2.aruco as aruco
import numpy as np

# Define parameters for ArUco marker detection
aruco_dict_type = cv2.aruco.DICT_6X6_250 # Change dictionary type if needed
marker_length = 0.05  # Marker length in meters (adjust as needed)
aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)

marker_id = 42
marker_size = 500  # Size in pixels
marker_image = cv2.aruco.generateImageMarker(aruco_dict, marker_id, marker_size)


In [44]:
# plt.imshow(marker_image, cmap='gray')

In [45]:

# Load the camera calibration parameters
# Replace with your calibration file values

# Initialize video capture

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Initialize ArUco dictionary and detector parameters
    aruco_dict = aruco.getPredefinedDictionary(aruco_dict_type)
    parameters = aruco.DetectorParameters()

    # Detect ArUco markers
    detector = aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, _ = detector.detectMarkers(gray)

    if ids is not None:
        # Draw detected markers
        frame = aruco.drawDetectedMarkers(frame, corners, ids)

        # Estimate pose of each marker
        for i in range(len(ids)):
            rvec, tvec, _ = aruco.estimatePoseSingleMarkers(corners[i], marker_length, cameraMatrix, dist)

            # Draw the axis for the marker
            frame = cv2.drawFrameAxes(frame, cameraMatrix , dist, rvec, tvec, marker_length / 2)

            # Print pose info
            # print(f"Marker ID: {ids[i][0]}")
            # print(f"Rotation Vector (rvec): {rvec}")
            # print(f"Translation Vector (tvec): {tvec}\n")

    # Display the frame
    cv2.imshow("ArUco Pose Estimation", frame)

    # Exit loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Marker ID: 42
Rotation Vector (rvec): [[[-0.02158233 -3.10887146 -0.18648459]]]
Translation Vector (tvec): [[[ 0.01157958 -0.0068337   0.28832976]]]

Marker ID: 42
Rotation Vector (rvec): [[[1.97849029 1.82721609 0.4800015 ]]]
Translation Vector (tvec): [[[ 0.00811612 -0.02530937  0.24041439]]]

Marker ID: 42
Rotation Vector (rvec): [[[1.9754057  1.84703336 0.48937392]]]
Translation Vector (tvec): [[[ 0.00435377 -0.02278855  0.22747376]]]

Marker ID: 42
Rotation Vector (rvec): [[[1.98034506 1.90135306 0.46308383]]]
Translation Vector (tvec): [[[ 0.00056299 -0.02107649  0.2243189 ]]]

Marker ID: 42
Rotation Vector (rvec): [[[1.96593087 1.95220353 0.42256899]]]
Translation Vector (tvec): [[[-0.00341673 -0.02023161  0.22488556]]]

Marker ID: 42
Rotation Vector (rvec): [[[1.95042724 1.99386231 0.41939589]]]
Translation Vector (tvec): [[[-0.006733   -0.01940802  0.22445243]]]

Marker ID: 42
Rotation Vector (rvec): [[[1.95991823 2.04760124 0.37323693]]]
Translation Vector (tvec): [[[-0.00953

KeyboardInterrupt: 

In [49]:
corners

(array([[[383.,  41.],
         [373., 243.],
         [217., 225.],
         [221.,  41.]]], dtype=float32),)